# PyCity Schools Analysis

* As a whole, charter schools out-perform district schools across all metrics, including average reading and math scores, reading and math pass rates, and overall pass rates. The top five performing schools are charter schools, while the bottom five performing schools are district schools.

* As a whole, smaller and medium sized schools (less than 2000 students) out-perform larger schools (greater than 2000 students) across all metrics.

* Schools that spend less money per student (615 dollars or below) tend to out-perform schools that spend more money per student (615 dollars or greater).

* At first glance, there does not appear to be a significant difference in math and reading test scores across the grade levels in both charter and district schools.
---

In [20]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = 'schools_complete.csv'
student_data_to_load = 'students_complete.csv'

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

school_data_complete.head()

Student ID       student_name gender grade        school_name  \
0           0       Paul Bradley      M   9th  Huang High School   
1           1       Victor Smith      M  12th  Huang High School   
2           2    Kevin Rodriguez      M  12th  Huang High School   
3           3  Dr. Richard Scott      M  12th  Huang High School   
4           4         Bonnie Ray      F   9th  Huang High School   

   reading_score  math_score  School ID      type  size   budget  
0             66          79          0  District  2917  1910635  
1             94          61          0  District  2917  1910635  
2             90          60          0  District  2917  1910635  
3             67          58          0  District  2917  1910635  
4             97          84          0  District  2917  1910635

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [21]:
#Total number of schools
total_schools = school_data['School ID'].count()

#Total number of students
total_students = student_data['Student ID'].count()

#Total budget
total_budget = school_data['budget'].sum()

#Average math and reading scores
avg_math_score = student_data['math_score'].mean()
avg_read_score = student_data['reading_score'].mean()

In [22]:
#Overall passing rate
math_percent = len(student_data[student_data['math_score'] >= 70])/total_students * 100
read_percent = len(student_data[student_data['reading_score'] >= 70])/total_students * 100
overall_percent = (math_percent + read_percent)/2

In [23]:
#Create dataframe to hold results
district_sum_df = pd.DataFrame({'Total Schools': [total_schools], 'Total Students': [total_students], 'Total Budget': [total_budget], \
                                'Average Math Score': [avg_math_score], 'Average Reading Score': [avg_read_score], \
                                'Math Pass Rate': [math_percent], 'Reading Pass Rate': [read_percent], \
                                'Overall Passing Rate': [overall_percent]})

district_sum_df_format = district_sum_df[['Total Students', 'Total Schools', 'Total Budget', 'Average Math Score', 'Average Reading Score', 'Math Pass Rate', 'Reading Pass Rate', 'Overall Passing Rate']]

district_sum_df_format = district_sum_df_format.style.format({'Total Budget': '${:,.2f}'.format, 'Average Math Score': '{:.2f}'.format, 'Average Reading Score': '{:.2f}'.format, 'Math Pass Rate': '{:.2f}%'.format, 'Reading Pass Rate': '{:.2f}%'.format, 'Overall Passing Rate': '{:.2f}%'.format})

district_sum_df_format

## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [24]:
#Group data by school name
school_names = school_data_complete.groupby('school_name')

#Group data by school type - use max since one school type per school
school_type = school_names['type'].max()

#Calculate total number of students
number_students = school_names['Student ID'].count()

#Calculate total school budget - use max since one budget per school
school_budget = school_names['budget'].max()

#Calculate per student budget
student_budget = school_budget/number_students

#Calculate average math and reading scores
avg_math = school_names['math_score'].mean()
avg_read = school_names['reading_score'].mean()

In [25]:
#Calculate math and reading passing rates
school_names_math = school_data_complete.loc[school_data_complete['math_score'] >= 70, :].groupby('school_name')
school_names_read = school_data_complete.loc[school_data_complete['reading_score'] >= 70,:].groupby('school_name')

math_pass_counts = school_names_math['Student ID'].count()
read_pass_counts = school_names_read['Student ID'].count()

math_pass_rates = (math_pass_counts/number_students) * 100
read_pass_rates = (read_pass_counts/number_students) * 100
overall_pass_rates = ((math_pass_rates + read_pass_rates)/2)

In [26]:
#Create dataframe to hold results
school_sum_df = pd.DataFrame({'School Type': school_type, 'Total Students': number_students, 'Total School Budget': school_budget, \
                                 'Per Student Budget': student_budget, 'Average Math Score': avg_math, 'Average Reading Score': avg_read, \
                                 'Math Pass Rate': math_pass_rates, 'Reading Pass Rate': read_pass_rates, 'Overall Passing Rate': overall_pass_rates})

school_sum_df_format = school_sum_df[['School Type', 'Total Students', 'Total School Budget', 'Per Student Budget', 'Average Math Score', \
                                     'Average Reading Score', 'Math Pass Rate', 'Reading Pass Rate', 'Overall Passing Rate']]

school_sum_df_format = school_sum_df_format.style.format({'Total Students': '{:,}'.format, 'Total School Budget': '${:,}'.format, 'Per Student Budget': '${:.2f}', \
                                                         'Average Math Score': '{:.2f}'.format, 'Average Reading Score': '{:.2f}'.format, 'Math Pass Rate': '{:.2f}%'.format, \
                                                          'Reading Pass Rate': '{:.2f}%'.format, 'Overall Passing Rate': '{:.2f}%'.format})

school_sum_df_format.index.name = "School"

school_sum_df_format

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [27]:
#Sort our current data frame
topschools_df = school_sum_df.sort_values("Overall Passing Rate", ascending = False)

topschools_df_format = topschools_df[['School Type', 'Total Students', 'Total School Budget', 'Per Student Budget', 'Average Math Score', \
                                     'Average Reading Score', 'Math Pass Rate', 'Reading Pass Rate', 'Overall Passing Rate']]

topschools_df_format = topschools_df_format.head().style.format({'Total Students': '{:,}'.format, 'Total School Budget': '${:,}'.format, 'Per Student Budget': '${:.2f}', \
                                                   'Average Math Score': '{:.2f}'.format, 'Average Reading Score': '{:.2f}'.format, 'Math Pass Rate': '{:.2f}%'.format, \
                                                   'Reading Pass Rate': '{:.2f}%'.format, 'Overall Passing Rate': '{:.2f}%'.format})

topschools_df_format.index.name = "School"

topschools_df_format

## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [28]:
#Sort data again!

bottomschools_df = school_sum_df.sort_values("Overall Passing Rate")

bottomschools_df_format = bottomschools_df[['School Type', 'Total Students', 'Total School Budget', 'Per Student Budget', 'Average Math Score', \
                                     'Average Reading Score', 'Math Pass Rate', 'Reading Pass Rate', 'Overall Passing Rate']]

bottomschools_df_format = bottomschools_df_format.head().style.format({'Total Students': '{:,}'.format, 'Total School Budget': '${:,}'.format, 'Per Student Budget': '${:.2f}', \
                                                   'Average Math Score': '{:.2f}'.format, 'Average Reading Score': '{:.2f}'.format, 'Math Pass Rate': '{:.2f}%'.format, \
                                                   'Reading Pass Rate': '{:.2f}%'.format, 'Overall Passing Rate': '{:.2f}%'.format})

bottomschools_df_format.index.name = "School"

bottomschools_df_format

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [29]:
#Split the data into groups, and calculate average grades
freshmath_avg = school_data_complete.loc[school_data_complete['grade'] == "9th"].groupby('school_name')['math_score'].mean()
sophmath_avg = school_data_complete.loc[school_data_complete['grade'] == "10th"].groupby('school_name')['math_score'].mean()
junmath_avg = school_data_complete.loc[school_data_complete['grade'] == "11th"].groupby('school_name')['math_score'].mean()
senmath_avg = school_data_complete.loc[school_data_complete['grade'] == "12th"].groupby('school_name')['math_score'].mean()

#Create data frame with averages, and format data
math_pergrade = pd.DataFrame({'9th': freshmath_avg, '10th': sophmath_avg, '11th': junmath_avg, '12th': senmath_avg})

math_pergrade_format = math_pergrade[['9th', '10th', '11th', '12th']]

math_pergrade_format = math_pergrade_format.style.format({'9th':'{:.2f}'.format, '10th': '{:.2f}'.format, '11th':'{:.2f}'.format, '12th': '{:.2f}'.format})

math_pergrade_format.index.name = "School"
                                                          
math_pergrade_format

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [30]:
#Split the data into groups, and calculate average grades
freshread_avg = school_data_complete.loc[school_data_complete['grade'] == "9th"].groupby('school_name')['reading_score'].mean()
sophread_avg = school_data_complete.loc[school_data_complete['grade'] == "10th"].groupby('school_name')['reading_score'].mean()
junread_avg = school_data_complete.loc[school_data_complete['grade'] == "11th"].groupby('school_name')['reading_score'].mean()
senread_avg = school_data_complete.loc[school_data_complete['grade'] == "12th"].groupby('school_name')['reading_score'].mean()

#Create data frame with averages, and format data
read_pergrade = pd.DataFrame({'9th': freshread_avg, '10th': sophread_avg, '11th': junread_avg, '12th': senread_avg})

read_pergrade_format = read_pergrade[['9th', '10th', '11th', '12th']]

read_pergrade_format = read_pergrade_format.style.format({'9th':'{:.2f}'.format, '10th': '{:.2f}'.format, '11th':'{:.2f}'.format, '12th': '{:.2f}'.format})

read_pergrade_format.index.name = "School"
                                                          
read_pergrade_format

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [31]:
#Create bins
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

school_data_complete['Spending Bins'] = pd.cut(school_data_complete['budget']/school_data_complete['size'], bins=spending_bins, labels=group_names)

In [32]:
#Group by spending
spending_groups = school_data_complete.groupby('Spending Bins')

#Perform calculations
math_avg_spending = spending_groups['math_score'].mean()
read_avg_spending = spending_groups['reading_score'].mean()

math_pass_spendingct = school_data_complete[school_data_complete['math_score'] >= 70].groupby('Spending Bins')['Student ID'].count()
read_pass_spendingct = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('Spending Bins')['Student ID'].count()

math_pass_spendingpct = 100 * (math_pass_spendingct/spending_groups['Student ID'].count())
read_pass_spendingpct = 100 * (read_pass_spendingct/spending_groups['Student ID'].count())

overall_pass_spending = (math_pass_spendingpct + read_pass_spendingpct)/2

In [33]:
#Create and format data frame
spending_df = pd.DataFrame({'Average Math Score': math_avg_spending, 'Average Reading Score': read_avg_spending, 'Math Pass Rate': math_pass_spendingpct, \
                           'Reading Pass Rate': read_pass_spendingpct, 'Overall Passing Rate': overall_pass_spending})

spending_df_format = spending_df[['Average Math Score', 'Average Reading Score', 'Math Pass Rate', 'Reading Pass Rate', 'Overall Passing Rate']]

spending_df_format = spending_df_format.style.format({'Average Math Score': '{:.2f}'.format, 'Average Reading Score': '{:.2f}'.format, 'Math Pass Rate': '{:.2f}%'.format, \
                                                      'Reading Pass Rate': '{:.2f}%'.format, 'Overall Passing Rate': '{:.2f}%'.format})

spending_df_format

## Scores by School Size

* Perform the same operations as above, based on school size.

In [34]:
#Create bins
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

school_data_complete['Size Bins'] = pd.cut(school_data_complete['size'], bins=size_bins, labels=group_names)

In [35]:
#Group by size
size_groups = school_data_complete.groupby('Size Bins')

#Perform calculations
math_avg_size = size_groups['math_score'].mean()
read_avg_size = size_groups['reading_score'].mean()

math_pass_sizect = school_data_complete[school_data_complete['math_score'] >= 70].groupby('Size Bins')['Student ID'].count()
read_pass_sizect = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('Size Bins')['Student ID'].count()

math_pass_sizepct = 100 * (math_pass_sizect/size_groups['Student ID'].count())
read_pass_sizepct = 100 * (read_pass_sizect/size_groups['Student ID'].count())

overall_pass_size = (math_pass_sizepct + read_pass_sizepct)/2

In [36]:
#Create and format data frame
size_df = pd.DataFrame({'Average Math Score': math_avg_size, 'Average Reading Score': read_avg_size, 'Math Pass Rate': math_pass_sizepct, \
                           'Reading Pass Rate': read_pass_sizepct, 'Overall Passing Rate': overall_pass_size})

size_df_format = size_df[['Average Math Score', 'Average Reading Score', 'Math Pass Rate', 'Reading Pass Rate', 'Overall Passing Rate']]

size_df_format = size_df_format.style.format({'Average Math Score': '{:.2f}'.format, 'Average Reading Score': '{:.2f}'.format, 'Math Pass Rate': '{:.2f}%'.format, \
                                              'Reading Pass Rate': '{:.2f}%'.format, 'Overall Passing Rate': '{:.2f}%'.format})

size_df_format

## Scores by School Type

* Perform the same operations as above, based on school type.

In [37]:
#Group by type
type_groups = school_data_complete.groupby('type')

#Perform calculations
math_avg_type = type_groups['math_score'].mean()
read_avg_type = type_groups['reading_score'].mean()

math_pass_typect = school_data_complete[school_data_complete['math_score'] >= 70].groupby('type')['Student ID'].count()
read_pass_typect = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('type')['Student ID'].count()

math_pass_typepct = 100 * (math_pass_typect/type_groups['Student ID'].count())
read_pass_typepct = 100 * (read_pass_typect/type_groups['Student ID'].count())

overall_pass_type = (math_pass_typepct + read_pass_typepct)/2

In [38]:
#Create and format data frame
type_df = pd.DataFrame({'Average Math Score': math_avg_type, 'Average Reading Score': read_avg_type, 'Math Pass Rate': math_pass_typepct, \
                           'Reading Pass Rate': read_pass_typepct, 'Overall Passing Rate': overall_pass_type})

type_df_format = type_df[['Average Math Score', 'Average Reading Score', 'Math Pass Rate', 'Reading Pass Rate', 'Overall Passing Rate']]

type_df_format = type_df_format.style.format({'Average Math Score': '{:.2f}'.format, 'Average Reading Score': '{:.2f}'.format, 'Math Pass Rate': '{:.2f}%'.format, \
                                              'Reading Pass Rate': '{:.2f}%'.format, 'Overall Passing Rate': '{:.2f}%'.format})

type_df_format